# Run Delphes and extract observables

Johann Brehmer, Kyle Cranmer, Felix Kling, Duccio Pappadopulo, Josh Ruderman 2018

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
% matplotlib inline
import logging
import os
import math
from collections import OrderedDict

from madminer.delphes import DelphesProcessor
from madminer.sampling import combine_and_shuffle
from madminer.utils.particle import MadMinerParticle

logging.basicConfig(format='%(asctime)s  %(message)s', datefmt='%H:%M')


In [2]:
base_dir = '/Users/johannbrehmer/work/projects/madminer/diboson_mining/'
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/'

In [3]:
sample_dir = base_dir + 'data/samples/wgamma/'
card_dir = base_dir + 'cards/wgamma/'
ufo_model_dir = card_dir + 'SMWgamma_UFO'
run_card_dir = card_dir + 'run_cards/'
mg_process_dir = base_dir + 'data/mg_processes/wgamma/'
log_dir = base_dir + 'logs/wgamma/'
temp_dir = base_dir + 'data/temp'
delphes_dir = mg_dir + 'Delphes'

## Observables and cuts

In [4]:
def resurrection_phi(leptons, photons, jets, met):
    # Parameters
    mw = 80.4
    
    # Particles
    if len(leptons) < 1 or len(photons) < 1:
        raise RuntimeError()
    
    l = leptons[0]
    a = photons[0]
    
    # Transverse mass and Delta
    mt = ((l.pt + met.pt)**2 - (l+met).pt**2)**0.5
    deltasq = 0.
    if met.pt > 0. and l.pt > 0.:
        deltasq = (mw**2 - mt**2) / (2. * met.pt * l.pt)
    
    # v reconstruction, "normal" case
    if deltasq > 0.:
        # Two solutions
        temp = np.log(1 + deltasq**0.5 * (2 + deltasq)**0.5 + deltasq)
        eta_v_plus = l.eta + temp
        eta_v_minus = l.eta - temp
        
        # Randomly select one of them
        dice = np.random.rand()
        if dice > 0.5:
            eta_v = eta_v_plus
        else:
            eta_v = eta_v_minus
            
    # v reconstruction, "other" case
    else:
        eta_v = l.eta
        
    # v particle
    v = MadMinerParticle()
    v.setptetaphim(met.pt, eta_v, met.phi(), 0.)
    
    # W and Wgamma reconstruction
    w = l + v
    vv = w + a
    
    # Boost into VV frame
    v_ = v.boost(vv.boostvector)
    l_ = l.boost(vv.boostvector)
    a_ = a.boost(vv.boostvector)
    w_ = w.boost(vv.boostvector)
    r_ = vv.boost(vv.boostvector)

    # Calculate axes of "special frame" (1708.07823)
    z_ = w_.vector.unit()
    x_ = (r_.vector - z_ * r_.vector.dot(z_)).unit()
    y_ = z_.cross(x_)
    
    # Calculate x and y components of lepton wrt special x_, y_, z_ system
    lx_ = l.vector.dot(x_)
    ly_ = l.vector.dot(y_)
    
    # Calculate phi
    phi = math.atan2(ly_, lx_)
    
    return phi
    

In [5]:
# Number of particles considered
n_leptons = 1
n_photons = 1
n_jets = 1

def setup_observables(delphesprocessor):
    delphesprocessor.observables = OrderedDict()
    delphesprocessor.observables_required = OrderedDict()
    delphesprocessor.observables_defaults = OrderedDict()
        
    # Default observables
    delphesprocessor.add_default_observables(
        n_leptons_max=n_leptons,
        n_photons_max=n_photons,
        n_jets_max=n_jets
    )

    # Correlations with MET
    for s1, i1 in [('l', 0), ('a', 0)]:
        delphesprocessor.add_observable(
            'deltaphi_{}{}_met'.format(s1, i1+1),
            '{}[{}].phi() - met.phi()'.format(s1, i1),
            required=True
        )

    # Reconstructed W
    delphesprocessor.add_observable(
        'm_l1_met',
        '(l[0] + met).m',
        required=True
    )
    delphesprocessor.add_observable(
        'pt_l1_met',
        '(l[0] + met).pt',
        required=True
    )

    # Selected correlations between particles
    for s1, i1, s2, i2 in [('l', 0, 'a', 0)]:
        delphesprocessor.add_observable(
            'm_{}{}_{}{}'.format(s1, i1+1, s2, i2+1),
            '({}[{}] + {}[{}]).m'.format(s1, i1, s2, i2),
            required=True
        )
        delphesprocessor.add_observable(
            'deltaeta_{}{}_{}{}'.format(s1, i1+1, s2, i2+1),
            '{}[{}].eta - {}[{}].eta'.format(s1, i1, s2, i2),
            required=True
        )
        delphesprocessor.add_observable(
            'deltaphi_{}{}_{}{}'.format(s1, i1+1, s2, i2+1),
            '{}[{}].phi() - {}[{}].phi()'.format(s1, i1, s2, i2),
            required=True
        )

    # Wgamma system
    delphesprocessor.add_observable(
        'm_a1_l1_met',
        '(a[0] + l[0] + met).m',
        required=True
    )
    delphesprocessor.add_observable(
        'pt_a1_l1_met',
        '(a[0] + l[0] + met).pt',
        required=True
    )
    
    # Resurection phi
    delphesprocessor.add_observable_from_function(
        'phi_resurrection',
        resurrection_phi,
        required=True
    )


In [6]:
def setup_cuts(delphesprocessor, use_tight_cuts=False):
    delphesprocessor.cuts = []
    delphesprocessor.cuts_default_pass = []
        
    delphesprocessor.add_cut(
        'n_ls > 0.1'
    )
    delphesprocessor.add_cut(
        'n_as > 0.1'
    )
    if use_tight_cuts:
        delphesprocessor.add_cut(
            'pt_a1 >= 300.'
        )
        delphesprocessor.add_cut(
            'pt_l1 >= 80.'
        )
        delphesprocessor.add_cut(
            'et_miss >= 80.'
        )
        delphesprocessor.add_cut(
            '(deltaphi_l1_a1**2 + deltaeta_l1_a1**2)**0.5 >= 3.'
        )
        delphesprocessor.add_cut(
            'eta_l1**2 < 2.4**2'
        )
        
    else:
        delphesprocessor.add_cut(
            'et_miss >= 10.'
        )

## Main loop

In [7]:
n_runs_per_benchmark = 18  # Number of run_cards

In [12]:
run = 0

for i_card in range(n_runs_per_benchmark):
    logging.info('Starting analysis of runs for card {}'.format(i_card))
            
    # Load setup
    dp = DelphesProcessor(sample_dir + 'setup.h5', debug=False)
    
    # Load events
    for benchmark in ['sm']:
        run += 1
        run_str = str(run)
        if len(run_str) < 2:
            run_str = '0' + run_str

        if run < 15:
            dp.add_hepmc_sample(
                mg_process_dir + 'Events/run_{}/tag_2_pythia8_events.hepmc.gz'.format(run_str),
                sampled_from_benchmark=benchmark
            )
        else:
            dp.add_hepmc_sample(
                mg_process_dir + 'Events/run_{}/tag_1_pythia8_events.hepmc.gz'.format(run_str),
                sampled_from_benchmark=benchmark
            )
        
    # Run Delphes
    dp.run_delphes(
        delphes_directory=delphes_dir,
        delphes_card=card_dir + 'delphes_card.dat',
        log_directory=log_dir,
        initial_command='source activate python2',
    )
    
    # Set up observables and cuts
    setup_observables(dp)
    setup_cuts(dp)
    
    # Analyse Delphes sample
    dp.analyse_delphes_samples(delete_delphes_files=True)
        
    # Save
    dp.save(sample_dir + 'samples_{}.h5'.format(i_card))


21:45  Starting analysis of runs for card 14
21:45  Adding HepMC sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_15/tag_1_pythia8_events.hepmc.gz
21:46  Running Delphes (/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/Delphes) on event sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_15/tag_1_pythia8_events.hepmc.gz
21:51  Analysing Delphes sample /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_15/tag_1_pythia8_events_delphes.root
21:55    Requiring existence of observable et_miss: 150000 events pass, 0 events removed
21:55    Requiring existence of observable phi_miss: 150000 events pass, 0 events removed
21:55    Requiring existence of observable e_visible: 150000 events pass, 0 events removed
21:55    Requiring existence of observable eta_visible: 150000 events pass, 0 events removed
21:55    Requiring existence of

22:21  Analysing Delphes sample /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_18/tag_1_pythia8_events_delphes.root
22:24    Requiring existence of observable et_miss: 150000 events pass, 0 events removed
22:24    Requiring existence of observable phi_miss: 150000 events pass, 0 events removed
22:24    Requiring existence of observable e_visible: 150000 events pass, 0 events removed
22:24    Requiring existence of observable eta_visible: 150000 events pass, 0 events removed
22:24    Requiring existence of observable n_ls: 150000 events pass, 0 events removed
22:24    Requiring existence of observable n_as: 150000 events pass, 0 events removed
22:24    Requiring existence of observable n_js: 150000 events pass, 0 events removed
22:24    Requiring existence of observable deltaphi_l1_met: 122305 events pass, 27695 events removed
22:24    Requiring existence of observable deltaphi_a1_met: 140667 events pass, 9333 events removed
22:24    Requi

## Combine samples

In [13]:
filenames_in = [sample_dir + 'samples_{}.h5'.format(i_card) for i_card in range(n_runs_per_benchmark)]

combine_and_shuffle(filenames_in, sample_dir + 'samples.h5')

22:24  Careful: this tool assumes that all samples are generated with the same setup, including identical benchmarks (and thus morphing setup). If it is used with samples with different settings, there will be wrong results! There are no explicit cross checks in place yet.
22:24  Copying setup from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_0.h5 to /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples.h5
22:24  Loading samples from file 1 / 18 at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_0.h5
22:24  Loading samples from file 2 / 18 at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_1.h5
22:24  Loading samples from file 3 / 18 at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_2.h5
22:24  Loading samples from file 4 / 18 at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/sam

## Main loop  with tight cuts

In [ ]:
run = 0

for i_card in range(n_runs_per_benchmark):
    logging.info('Starting analysis of runs for card {}'.format(i_card))
            
    # Load setup
    dp = DelphesProcessor(sample_dir + 'setup.h5', debug=False)
    
    # Load events
    for benchmark in ['sm']:
        run += 1
        run_str = str(run)
        if len(run_str) < 2:
            run_str = '0' + run_str

        if run < 15:
            dp.add_hepmc_sample(
                mg_process_dir + 'Events/run_{}/tag_2_pythia8_events.hepmc.gz'.format(run_str),
                sampled_from_benchmark=benchmark
            )
        else:
            dp.add_hepmc_sample(
                mg_process_dir + 'Events/run_{}/tag_1_pythia8_events.hepmc.gz'.format(run_str),
                sampled_from_benchmark=benchmark
            )
        
    # Run Delphes
    dp.run_delphes(
        delphes_directory=delphes_dir,
        delphes_card=card_dir + 'delphes_card.dat',
        log_directory=log_dir,
        initial_command='source activate python2',
    )
    
    # Set up observables and cuts
    setup_observables(dp)
    setup_cuts(dp, use_tight_cuts=True)
    
    # Analyse Delphes sample
    dp.analyse_delphes_samples(delete_delphes_files=True)
        
    # Save
    dp.save(sample_dir + 'samples_tight_{}.h5'.format(i_card))


16:18  
16:18  ------------------------------------------------------------
16:18  |                                                          |
16:18  |  MadMiner v2018.10.30                                    |
16:18  |                                                          |
16:18  |           Johann Brehmer, Kyle Cranmer, and Felix Kling  |
16:18  |                                                          |
16:18  ------------------------------------------------------------
16:18  
16:18  Adding HepMC sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_01/tag_2_pythia8_events.hepmc.gz
16:18  Running Delphes (/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/Delphes) on event sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_01/tag_2_pythia8_events.hepmc.gz
16:25  Analysing Delphes sample /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wg

16:53    Cut n_as > 0.1: 131348 events pass, 18652 events removed
16:53    Cut pt_a1 >= 300.: 38482 events pass, 111518 events removed
16:53    Cut pt_l1 >= 80.: 91662 events pass, 58338 events removed
16:53    Cut et_miss >= 80.: 89835 events pass, 60165 events removed
16:53    Cut (deltaphi_l1_a1**2 + deltaeta_l1_a1**2)**0.5 >= 3.: 80213 events pass, 69787 events removed
16:53    Cut eta_l1**2 < 2.4**2: 148200 events pass, 1800 events removed
16:53  Starting analysis of runs for card 3
16:53  Adding HepMC sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_04/tag_2_pythia8_events.hepmc.gz
16:54  Running Delphes (/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/Delphes) on event sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_04/tag_2_pythia8_events.hepmc.gz
16:58  Analysing Delphes sample /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/w

17:21    Cut pt_a1 >= 300.: 92870 events pass, 57130 events removed
17:21    Cut pt_l1 >= 80.: 111228 events pass, 38772 events removed
17:21    Cut et_miss >= 80.: 109964 events pass, 40036 events removed
17:21    Cut (deltaphi_l1_a1**2 + deltaeta_l1_a1**2)**0.5 >= 3.: 92593 events pass, 57407 events removed
17:21    Cut eta_l1**2 < 2.4**2: 148627 events pass, 1373 events removed
17:21  Starting analysis of runs for card 6
17:21  Adding HepMC sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_07/tag_2_pythia8_events.hepmc.gz
17:21  Running Delphes (/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/Delphes) on event sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_07/tag_2_pythia8_events.hepmc.gz
17:26  Analysing Delphes sample /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_07/tag_2_pythia8_events_delphes.root


## Combine samples with tight cuts

In [ ]:
filenames_in = [sample_dir + 'samples_tight_{}.h5'.format(i_card) for i_card in range(n_runs_per_benchmark)]

combine_and_shuffle(filenames_in, sample_dir + 'samples_tight.h5')